In [1]:
#Import library yang dibutuhkan
import re
import spacy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary
import string

In [3]:
#Membuka file data tweets
data_tweet = pd.read_csv("tweets-data/filtered_dataset_kurikulum_merdeka2.csv", encoding='UTF-8')

In [4]:
df = pd.DataFrame(data_tweet['full_text'])
df.head(10)

,full_text
0,Selamat hari guru jg buat calon guru yang cape...
1,@mediaindonesia Bukannya kurikulum merdeka mau...
2,@syahrindraaa @GhibahNaqMetal For Kurikulum Me...
3,Modul ajar ‍‍‍ Duhhh siapa yang udah dah perna...
4,Guru yang baik adalah guru yang merdeka dari k...
5,30 Soal PAS Bahasa Indonesia Kelas 3 Semester ...
6,@txtdrbahasa Bukan kurikulum merdeka nih
7,@n4jaemin kurikulum merdeka bukannya merdeka m...
8,@ikonypncfc iyaaa soalnya angkatan sekolahku p...
9,Tulisan ini merupakan refleksi atas pelaksanaa...


In [ ]:
def filtering_text(text):
    # Konversi teks ke lowercase
    text = text.lower()
    # Menghapus link
    text = re.sub(r'https?:\/\/\S+', '', text)
    # Menghapus username dan hashtag
    text = re.sub(r"[@#][A-Za-z0-9_]+", " ", text)
    # Menghapus karakter b' atau b''
    text = re.sub(r"(b'{1,2})", "", text)
    # Menghapus karakter non-alfabet
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Menghapus angka
    text = re.sub(r'\d+', '', text)
    # Menghapus tanda baca
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Menghapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text
 
df['filtered'] = df['full_text'].apply(filtering_text)
df.head(-5)

,full_text,filtered
0,Selamat hari guru jg buat calon guru yang cape...,selamat hari guru jg buat calon guru yang cape...
1,@mediaindonesia Bukannya kurikulum merdeka mau...,bukannya kurikulum merdeka mau diganti buu
2,@syahrindraaa @GhibahNaqMetal For Kurikulum Me...,for kurikulum merdeka
3,Modul ajar ‍‍‍ Duhhh siapa yang udah dah perna...,modul ajar duhhh siapa yang udah dah pernah bi...
4,Guru yang baik adalah guru yang merdeka dari k...,guru yang baik adalah guru yang merdeka dari k...
...,...,...
2255,Ganti menteri ganti kurikulum menjadi istilah...,ganti menteri ganti kurikulum menjadi istilah ...
2256,Mengganti Kurikulum Merdeka Menata Ulang Arah ...,mengganti kurikulum merdeka menata ulang arah ...
2257,@tekarok007 @BuzzerShestar Indonesia ini cocok...,indonesia ini cocoknya ada un udah terbukti ga...
2258,Kegiatan Penyerahan Sertifikat Pelaksana Kurik...,kegiatan penyerahan sertifikat pelaksana kurik...


In [6]:
#Proses stopwords dan stemming
def stop_stem(text):
    #stopword
    with open('kamus.txt') as kamus:
        word = kamus.readlines()
        list_stopword = [line.replace('\n',"") for line in word]
    dictionary = ArrayDictionary(list_stopword)
    stopword = StopWordRemover(dictionary)
    text = stopword.remove(text)
    # stemming
    factory_stemmer = StemmerFactory()
    stemmer = factory_stemmer.create_stemmer()
    text = stemmer.stem(text)
    return text

df['cleaned'] = df['filtered'].apply(stop_stem)

df.head(5)

,full_text,filtered,cleaned
0,Selamat hari guru jg buat calon guru yang cape...,selamat hari guru jg buat calon guru yang cape...,selamat hari guru jg calon guru capek bgt ma k...
1,@mediaindonesia Bukannya kurikulum merdeka mau...,bukannya kurikulum merdeka mau diganti buu,bukan kurikulum merdeka ganti buu
2,@syahrindraaa @GhibahNaqMetal For Kurikulum Me...,for kurikulum merdeka,for kurikulum merdeka
3,Modul ajar ‍‍‍ Duhhh siapa yang udah dah perna...,modul ajar duhhh siapa yang udah dah pernah bi...,modul ajar duhhh siapa udah pernah modul ajar ...
4,Guru yang baik adalah guru yang merdeka dari k...,guru yang baik adalah guru yang merdeka dari k...,guru baik adalah guru merdeka kurikulum bukan ...


In [ ]:
# Load spaCy model untuk Bahasa Indonesia
nlp = spacy.load('xx_ent_wiki_sm')

def tokenize_spacy(text):
    # Tokenisasi dengan spaCy
    doc = nlp(text)
    return [token.text for token in doc]

# Terapkan fungsi tokenize_spacy ke kolom 'cleaned'
df['tokenize'] = df['cleaned'].apply(tokenize_spacy)

# Tampilkan 5 data pertama
df.head(5)

,full_text,filtered,cleaned,tokenize
0,Selamat hari guru jg buat calon guru yang cape...,selamat hari guru jg buat calon guru yang cape...,selamat hari guru jg calon guru capek bgt ma k...,"[selamat, hari, guru, jg, calon, guru, capek, ..."
1,@mediaindonesia Bukannya kurikulum merdeka mau...,bukannya kurikulum merdeka mau diganti buu,bukan kurikulum merdeka ganti buu,"[bukan, kurikulum, merdeka, ganti, buu]"
2,@syahrindraaa @GhibahNaqMetal For Kurikulum Me...,for kurikulum merdeka,for kurikulum merdeka,"[for, kurikulum, merdeka]"
3,Modul ajar ‍‍‍ Duhhh siapa yang udah dah perna...,modul ajar duhhh siapa yang udah dah pernah bi...,modul ajar duhhh siapa udah pernah modul ajar ...,"[modul, ajar, duhhh, siapa, udah, pernah, modu..."
4,Guru yang baik adalah guru yang merdeka dari k...,guru yang baik adalah guru yang merdeka dari k...,guru baik adalah guru merdeka kurikulum bukan ...,"[guru, baik, adalah, guru, merdeka, kurikulum,..."


In [10]:
kamus_normalisasi = pd.read_csv("slang.csv")

kata_normalisasi_dict = {}

for index, row in kamus_normalisasi.iterrows():
    if row[0] not in kata_normalisasi_dict:
        kata_normalisasi_dict[row[0]] = row[1] 
        
def normalisasi_kata(document):
    return [kata_normalisasi_dict[term] if term in kata_normalisasi_dict else term for term in document]

df['normalisasi'] = df['tokenize'].apply(normalisasi_kata)

df.head(-5)

/var/folders/zh/42x7qv3n3dj722_0cqth5c0w0000gn/T/ipykernel_40209/623310452.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in kata_normalisasi_dict:
/var/folders/zh/42x7qv3n3dj722_0cqth5c0w0000gn/T/ipykernel_40209/623310452.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kata_normalisasi_dict[row[0]] = row[1]


,full_text,filtered,cleaned,tokenize,normalisasi
0,Selamat hari guru jg buat calon guru yang cape...,selamat hari guru jg buat calon guru yang cape...,selamat hari guru jg calon guru capek bgt ma k...,"[selamat, hari, guru, jg, calon, guru, capek, ...","[selamat, hari, guru, juga, calon, guru, kesal..."
1,@mediaindonesia Bukannya kurikulum merdeka mau...,bukannya kurikulum merdeka mau diganti buu,bukan kurikulum merdeka ganti buu,"[bukan, kurikulum, merdeka, ganti, buu]","[bukan, kurikulum, merdeka, ganti, buu]"
2,@syahrindraaa @GhibahNaqMetal For Kurikulum Me...,for kurikulum merdeka,for kurikulum merdeka,"[for, kurikulum, merdeka]","[untuk, kurikulum, merdeka]"
3,Modul ajar ‍‍‍ Duhhh siapa yang udah dah perna...,modul ajar duhhh siapa yang udah dah pernah bi...,modul ajar duhhh siapa udah pernah modul ajar ...,"[modul, ajar, duhhh, siapa, udah, pernah, modu...","[modul, ajar, duhhh, siapa, sudah, pernah, mod..."
4,Guru yang baik adalah guru yang merdeka dari k...,guru yang baik adalah guru yang merdeka dari k...,guru baik adalah guru merdeka kurikulum bukan ...,"[guru, baik, adalah, guru, merdeka, kurikulum,...","[guru, baik, adalah, guru, merdeka, kurikulum,..."
...,...,...,...,...,...
2255,Ganti menteri ganti kurikulum menjadi istilah...,ganti menteri ganti kurikulum menjadi istilah ...,ganti menteri ganti kurikulum jadi istilah pal...,"[ganti, menteri, ganti, kurikulum, jadi, istil...","[ganti, menteri, ganti, kurikulum, jadi, istil..."
2256,Mengganti Kurikulum Merdeka Menata Ulang Arah ...,mengganti kurikulum merdeka menata ulang arah ...,ganti kurikulum merdeka tata ulang arah didik ...,"[ganti, kurikulum, merdeka, tata, ulang, arah,...","[ganti, kurikulum, merdeka, tata, ulang, arah,..."
2257,@tekarok007 @BuzzerShestar Indonesia ini cocok...,indonesia ini cocoknya ada un udah terbukti ga...,indonesia cocok ada un udah bukti gaada un sis...,"[indonesia, cocok, ada, un, udah, bukti, gaada...","[indonesia, cocok, ada, un, sudah, bukti, tida..."
2258,Kegiatan Penyerahan Sertifikat Pelaksana Kurik...,kegiatan penyerahan sertifikat pelaksana kurik...,giat serah sertifikat laksana kurikulum merdek...,"[giat, serah, sertifikat, laksana, kurikulum, ...","[giat, serah, sertifikat, laksana, kurikulum, ..."


In [8]:
df.to_csv('tweets-data/preprocessed_dataset_kurikulum_merdeka2.csv', index=False)